NOW WE TRAIN AND VALIDATE ON ALL (ITEM, SHOP) EVER SEEN IN DATA
Shold be:
train - all (item ,shop) previosly seen
test - dot product of (item, shop) in current dbn

Now it is as it should be

In [3]:
import pandas as pd
import numpy as np


In [4]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')

In [5]:
test = pd.read_csv('../data_cleaned/test.csv')
test['date_block_num'] = 34

In [6]:
data_train = pd.concat([data_train,test ], ignore_index=True).drop('ID', axis=1).fillna(0)

In [7]:
data_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,06.01.2013,0,25,2554,1709.05,1.0
3,15.01.2013,0,25,2555,1099.00,1.0
4,10.01.2013,0,25,2564,349.00,1.0
...,...,...,...,...,...,...
3142680,0,34,45,18454,0.00,0.0
3142681,0,34,45,16188,0.00,0.0
3142682,0,34,45,15757,0.00,0.0
3142683,0,34,45,19648,0.00,0.0


In [8]:
def prepare_past_ID_s(data_train):
    data_train['shop_item'] = [tuple([shop, item]) for shop, item in zip(data_train['shop_id'], data_train['item_id'])]
    #34 block contains A LOT more shop_item than others
    shop_item_pairs_in_dbn = data_train.groupby('date_block_num')['shop_item'].apply(np.unique)
    data_train = data_train.drop(['shop_item'], axis=1)
    
    shop_item_pairs_WITH_PREV_in_dbn = shop_item_pairs_in_dbn.copy()
    
    print(np.array(shop_item_pairs_WITH_PREV_in_dbn.index))
    arr = np.array(shop_item_pairs_WITH_PREV_in_dbn.index)
    for block in arr[arr>=0]:
        if block == 0:
            continue
        shop_item_pairs_WITH_PREV_in_dbn[block] = np.unique(np.append(shop_item_pairs_WITH_PREV_in_dbn[block -1],
                                                            shop_item_pairs_in_dbn[block-1]))
        print(len(shop_item_pairs_WITH_PREV_in_dbn[block]))

    return shop_item_pairs_in_dbn, shop_item_pairs_WITH_PREV_in_dbn

In [9]:
shop_item_pairs_in_dbn, shop_item_pairs_WITH_PREV_in_dbn = prepare_past_ID_s(data_train)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]
63170
97186
126662
146380
162679
178831
192610
205423
216508
228547
240348
255924
263803
271937
281288
290301
299290
307259
314673
322344
330092
338979
349486
362849
368644
374512
380928
386411
391327
396282
401230
406603
411762
418882


In [10]:
pd.concat([shop_item_pairs_WITH_PREV_in_dbn.map(len),shop_item_pairs_in_dbn.map(len)], axis=1)

,shop_item,shop_item
date_block_num,,
0,63170,63170
1,63170,59911
2,97186,63951
3,126662,54590
4,146380,53276
5,162679,56173
6,178831,58004
7,192610,57988
8,205423,51543


In [11]:
from sklearn.metrics import root_mean_squared_error

In [12]:
from collections import defaultdict

In [13]:
def make_X_lag_format(data, dbn):
    """
    transform X to lag format
    columns with dbn in names become lag_0, dbn-1 - lag_1 etc.
    """
    
    lag_cols = defaultdict()
    for col in data.columns:
        
        if ''.join(re.findall(r'\d+', col)) == '':
                continue
        
        split = filter(None, re.split(r'(\d+)', col))
        split=list(split)
        #print(split)
        if len(split)==2:    
            lag_cols[col] = split[0] + '_lag_' + str(dbn - int(split[1]))

    #print(lag_cols)
    data = data.rename(columns=dict(lag_cols))
    #print(data.columns)
    return data

In [14]:
def prepare_train(data, valid ):
    """
    returns one batch of merged data with required IDs from valid
    """
    #print(data)
    valid_shop_item = valid
    valid_shop_item = list(zip(*valid_shop_item))
    df = pd.DataFrame({'item_id':valid_shop_item[1],'shop_id':valid_shop_item[0]} )
    data = df.merge(data, on=['shop_id','item_id'], how='left').fillna(0)
    
    return data

In [15]:

def prepare_val(data, valid ):
    """
    returns one batch of merged data with required IDs from valid
    """
    
    df = pd.DataFrame({'item_id':valid[:,1],'shop_id':valid[:,0]} )
    data = df.merge(data, on=['shop_id','item_id'], how='left').fillna(0)
    return data

In [16]:
import re


In [17]:
def prepare_data_train_boosting(data, valid, dbn):
    """
    
    """
    train = prepare_train (data, valid)
    lag_cols = []
    for col in data.columns:
        
        if ''.join(re.findall(r'\d+', col)) == '':
                lag_cols.append(col)
                continue
        #if 'shop_item_cnt' not in col:
        #    continue
            
        for db in range(0,dbn-1):
            
            if db == int(''.join(re.findall(r'\d+', col))):
                lag_cols.append(col)

    X = train[lag_cols]
    Y = train[f'shop_item_cnt{dbn-1}']
    
    return X, Y
        

In [18]:
def prepare_data_validation_boosting(data, valid, dbn):
    """
    
    """
    test = prepare_val (data, valid)
    
    lag_cols = []
    for col in test.columns:
        
            
        if ''.join(re.findall(r'\d+', col)) == '':
                lag_cols.append(col)
                continue
        #if 'shop_item_cnt' not in col:
        #    continue
        for db in range(1,dbn):
            
            if db == int(''.join(re.findall(r'\d+', col))):
                #print(db, int(''.join(re.findall(r'\d+', col))))
                lag_cols.append(col)

    X = test[lag_cols]
    Y = test[f'shop_item_cnt{dbn}']
    
    return X, Y

In [19]:
np.random.permutation([[1,2],[3,4]])

array([[1, 2],
       [3, 4]])

In [20]:
def create_batch_train(merged, batch_size, dbn):
    """
    
    """
    #merged = pd.read_csv('data/merged.csv', chunksize=500000)
    #merged = pd.read_csv('data/merged.csv')
    train = np.random.permutation (shop_item_pairs_WITH_PREV_in_dbn[dbn])
    chunck_num = (len(train)  // batch_size) + 1
    
    for idx in range(chunck_num):#split shop_item_pairs_WITH_PREV_in_dbn into chuncks
        #for chunck in merged:#split merged into chuncks
        train_ret = prepare_data_train_boosting(merged,train[idx*batch_size:(idx+1)*batch_size], dbn)
       
        if  train_ret[0].empty:
            yield [None, None]
        
        yield train_ret#, test

In [21]:
def create_batch_val(merged, batch_size, dbn):
    """
    
    """
    #merged = pd.read_csv('data/merged.csv', chunksize=500000) - (DOESNT WORK PROPERLY))))) - use it if merged doesnt fit memory
    #merged = pd.read_csv('data/merged.csv')
    val = shop_item_pairs_in_dbn[dbn]

    shops = np.unique(list(zip(*val))[0])
    items = np.unique(list(zip(*val))[1])

    cartesian_product = np.random.permutation (np.array(np.meshgrid(shops, items)).T.reshape(-1, 2))
    
    chunck_num = (len(cartesian_product)  // batch_size) + 1
    for idx in range(chunck_num):
        #for chunck in merged:
        train_ret = prepare_data_validation_boosting(merged,cartesian_product[idx*batch_size:(idx+1)*batch_size], dbn)
        #When in batches idx no elements that are in (shop, item) in batch of merged
        if  train_ret[0].empty:
            
            yield [None, None]
        #print(len(train_ret))
        
        yield train_ret#, test

In [22]:
names= [
       'ema_6_item_cnt_month_item_category_id_cat_shop_id',
       'ema_6_item_cnt_month_item_id',
       'shop_item_date_block_num_diff',
       'item_price',
       'shop_item_price',
       'item_price_change',
       'shop_item_price_change',
       'shop_category_cnt',
       'cnt_changes_items',
       'item_cnt',
       'cnt_changes_items_shops',]


In [23]:
def select_columns(X_train, dbn):
    X_train = append_some_columns(X_train,dbn)
    shop_item_cnt_lags= [1,2,3,4,5,6,7,8,9,10,11,12,24]
    cols=[]

    
    for col in X_train.columns:
        
        num = ''.join(re.findall(r'\d+', col))
        if col == 'item_id':
            continue
        if num == '':
            cols.append(col)
            continue
        num=int(num)
        
        if 'shop_item_cnt' in col:
            if num in shop_item_cnt_lags:
                cols.append(col)
                continue
        
        if 'diff' in col:
            if num <= 1:
                cols.append(col)

        if 'price_change' in col:
            if num <= 2:
                cols.append(col)

    

    
        
    return X_train[cols]
            

In [24]:
def append_some_columns(X_train, dbn):
    X_train['date_block_num'] = dbn
    X_train['month'] = dbn%12
    return X_train

In [25]:
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

In [26]:
"""
model=xgb.XGBRegressor(tree_method="hist",
                       #early_stopping_rounds=15,
                       enable_categorical=True,
                       max_depth = 7,
                       n_estimators = 70,
                       learning_rate = 0.05
                       )
"""

#model =RandomForestRegressor(max_depth = 11, n_estimators = 70)
model = LGBMRegressor(verbose=-1)

In [34]:
def train_model(model, merged,batch_size, val_month):
    
    first=True
    rmse = 0
    c=0
    columns_order=None
    
    Y_true_l = []
    preds_l = []
    for X_train,Y_train  in create_batch_train(merged,batch_size, val_month):
        X_train['shop_id'] = X_train['shop_id'].astype('category')
        X_train['item_category_id'] = X_train['item_category_id'].astype('category')
        X_train['city'] = X_train['city'].astype('category')
        X_train['super_category'] = X_train['super_category'].astype('category')
        #
        if X_train is None:
            print('None')
            continue
            
        Y_train = np.clip(Y_train,0,20)
        
        if X_train.empty:
            print('None')
            continue
        
        X_train = make_X_lag_format(X_train, val_month-1)
        
        X_train=select_columns(X_train, val_month-1)
        
        columns_order=X_train.columns
        
        
        if first:
            model.fit(X_train, Y_train)
            first=False
        else:
            model.fit(X_train, Y_train, init_model=model)
        
        if c==0:
            print('train columns by feature importnce',
                  model.feature_names_in_[np.argsort(model.feature_importances_)][::-1])
        y_train_pred = model.predict(X_train, validate_features=True)#lgb - validate features
        
        
        Y_true_l.append(Y_train)
        preds_l.append(y_train_pred)
        
        c+=1
        
    train_rmse = root_mean_squared_error(pd.concat(Y_true_l), np.concat(preds_l))
    print('train_rmse, ',train_rmse)
           

    return model, columns_order

In [35]:
def validate_model(model,merged,batch_size, val_month, columns_order):
    rmse = 0
    c=0
    
    val_preds = []
    Y_true_l = []
    preds_l = []
    #create_batch_train(merged,batch_size, val_month) - return train set, where Y_val
    #is shop_item_cnt_month{val_month}
    for X_val, Y_val in create_batch_val(merged,batch_size, val_month):#but then cartesian product used
        #X_val['city'] = X_val['city'].astype('category')
        #X_val['super_category'] = X_val['city'].astype('category')
        X_val['shop_id'] = X_val['shop_id'].astype('category')
        X_val['item_category_id'] = X_val['item_category_id'].astype('category')
        X_val['city'] = X_val['city'].astype('category')
        X_val['super_category'] = X_val['super_category'].astype('category')
        if X_val is None:
            continue
            
        Y_val = np.clip(Y_val,0,20)
        
        
        X_val = make_X_lag_format(X_val, val_month)
        
        X_val=select_columns(X_val, val_month)
        X_val = X_val[columns_order]

        if c==0:
            #print('train columns',X_val.columns)
            print(X_val.date_block_num)
        y_val_pred = model.predict(X_val, validate_features=True)#lgb - validate features
        y_val_pred = np.clip(y_val_pred,0,20)

        
        
        
        preds_l.append(y_val_pred)
        Y_true_l.append(Y_val)
        
        c+=1
        
        val_preds.append(y_val_pred)

    
    val_rmse = root_mean_squared_error(pd.concat(Y_true_l), np.concat(preds_l))
    print('val rmse, ',val_rmse)

    return val_preds, val_rmse

In [29]:
def validate_ML(merged, model):
    """
    Function for validating model
    
    """
    
    val_errors = []
    batch_size=20000
    val_preds=[]
    
    
    for val_month in range(22, 34):
        print('date_block_num', val_month)
        print('month', val_month%12)

        model,columns_order = train_model(model, merged,batch_size, val_month)
        
        val_pred, val_error = validate_model(model,merged,batch_size, val_month,columns_order)
        
        val_errors.append(val_error)
        val_preds.append(val_pred)
        

    return val_errors, val_preds

In [30]:
chunksize = 50000
l=[]
with pd.read_csv('data/merged.csv', chunksize=chunksize) as reader:
    for chunk in reader:
        l.append(chunk)

merged = pd.concat(l)

In [31]:
list(merged.columns)

['shop_id',
 'item_id',
 'shop_item_cnt0',
 'shop_item_cnt1',
 'shop_item_cnt2',
 'shop_item_cnt3',
 'shop_item_cnt4',
 'shop_item_cnt5',
 'shop_item_cnt6',
 'shop_item_cnt7',
 'shop_item_cnt8',
 'shop_item_cnt9',
 'shop_item_cnt10',
 'shop_item_cnt11',
 'shop_item_cnt12',
 'shop_item_cnt13',
 'shop_item_cnt14',
 'shop_item_cnt15',
 'shop_item_cnt16',
 'shop_item_cnt17',
 'shop_item_cnt18',
 'shop_item_cnt19',
 'shop_item_cnt20',
 'shop_item_cnt21',
 'shop_item_cnt22',
 'shop_item_cnt23',
 'shop_item_cnt24',
 'shop_item_cnt25',
 'shop_item_cnt26',
 'shop_item_cnt27',
 'shop_item_cnt28',
 'shop_item_cnt29',
 'shop_item_cnt30',
 'shop_item_cnt31',
 'shop_item_cnt32',
 'shop_item_cnt33',
 'shop_item_cnt34',
 'item_category_id',
 'city',
 'super_category',
 'ema_6_item_cnt_month_item_category_id_cat_shop_id6',
 'ema_6_item_cnt_month_item_category_id_cat_shop_id7',
 'ema_6_item_cnt_month_item_category_id_cat_shop_id8',
 'ema_6_item_cnt_month_item_category_id_cat_shop_id9',
 'ema_6_item_cnt_

In [32]:
merged.item_price_change10.describe()

count    521611.000000
mean        276.449389
std         810.369060
min           0.000000
25%           0.000000
50%           0.000000
75%         289.400000
max       18195.006619
Name: item_price_change10, dtype: float64

In [36]:
val_errors, val_preds = validate_ML(merged, model)

date_block_num 22
month 10
train columns by feature importnce ['shop_item_cnt_lag_1' 'item_price_change_lag_1'
 'shop_item_date_block_num_diff_lag_1' 'shop_id' 'item_category_id'
 'item_price_change_lag_2' 'shop_item_price_change_lag_1'
 'shop_item_cnt_lag_2' 'shop_item_cnt_lag_4' 'shop_item_cnt_lag_3'
 'shop_item_cnt_lag_5' 'shop_item_cnt_lag_6' 'shop_item_cnt_lag_12'
 'shop_item_cnt_lag_7' 'shop_item_cnt_lag_10' 'city'
 'shop_item_cnt_lag_11' 'shop_item_price_change_lag_2'
 'shop_item_cnt_lag_8' 'shop_item_cnt_lag_9' 'super_category' 'month'
 'date_block_num']
train_rmse,  0.780299250736004
0        22
1        22
2        22
3        22
4        22
         ..
19995    22
19996    22
19997    22
19998    22
19999    22
Name: date_block_num, Length: 20000, dtype: int64
val rmse,  1.2512429952309432
date_block_num 23
month 11
train columns by feature importnce ['shop_item_cnt_lag_1' 'item_category_id' 'item_price_change_lag_1'
 'shop_id' 'shop_item_date_block_num_diff_lag_1' 'shop_ite

KeyboardInterrupt: 

In [63]:
np.array(val_errors).mean()

np.float64(1.0513793847967667)

In [32]:
def create_submission(model,merged,batch_size, columns_order):
    val_month = 34
    test = pd.read_csv('../data_cleaned/test.csv')
    
    data_test = test
    PREDICTION = pd.DataFrame(columns=['shop_id','item_id','item_cnt_month'])
    
    for X_val, Y_val in create_batch_val(merged,batch_size, val_month):
            
        if X_val is None:
            continue
            
        Y_val = np.clip(Y_val,0,20)
        
        if X_val.empty:
            print('None')
            continue
            
        
        X_val = make_X_lag_format(X_val, val_month)
        X_val=select_columns(X_val, val_month)
        X_val = X_val[columns_order]

        
        y_val_pred=model.predict(X_val, validate_feature=True)
        y_val_pred = np.clip(y_val_pred,0,20)#lgb - validate features
        
        
        
        app = pd.DataFrame({'item_id':X_val.item_id,'shop_id': X_val.shop_id, 'item_cnt_month':y_val_pred})
        PREDICTION = pd.concat([PREDICTION, app],ignore_index=True)


    
    data_test = data_test.merge(PREDICTION,on=['shop_id','item_id'])[['ID','item_cnt_month']]
    return data_test
    

In [30]:
def create_submission_pipeline(merged):
    val_errors = []
    batch_size=50000
    val_errors=[]

    model = LGBMRegressor(verbose=-1)

    model,columns_order = train_model(model, merged,batch_size, 34)
    
    print('Feature importnaces in lgb:')
    print(model.feature_names_in_[np.argsort(model.feature_importances_)][::-1])
    
    data_test = create_submission(model,merged,batch_size,columns_order)

    return data_test

In [33]:
submission = create_submission_pipeline(merged)

train columns Index(['shop_id', 'item_id', 'shop_item_cnt_lag_32', 'shop_item_cnt_lag_31',
       'shop_item_cnt_lag_30', 'shop_item_cnt_lag_29', 'shop_item_cnt_lag_28',
       'shop_item_cnt_lag_27', 'shop_item_cnt_lag_26', 'shop_item_cnt_lag_25',
       'shop_item_cnt_lag_24', 'shop_item_cnt_lag_23', 'shop_item_cnt_lag_22',
       'shop_item_cnt_lag_21', 'shop_item_cnt_lag_20', 'shop_item_cnt_lag_19',
       'shop_item_cnt_lag_18', 'shop_item_cnt_lag_17', 'shop_item_cnt_lag_16',
       'shop_item_cnt_lag_15', 'shop_item_cnt_lag_14', 'shop_item_cnt_lag_13',
       'shop_item_cnt_lag_12', 'shop_item_cnt_lag_11', 'shop_item_cnt_lag_10',
       'shop_item_cnt_lag_9', 'shop_item_cnt_lag_8', 'shop_item_cnt_lag_7',
       'shop_item_cnt_lag_6', 'shop_item_cnt_lag_5', 'shop_item_cnt_lag_4',
       'shop_item_cnt_lag_3', 'shop_item_cnt_lag_2', 'shop_item_cnt_lag_1',
       'item_category_id', 'date_block_num', 'month'],
      dtype='object')
train_rmse,  0.5810684202416754
Feature importnaces

/tmp/ipykernel_8079/3946748697.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  PREDICTION = pd.concat([PREDICTION, app],ignore_index=True)


In [34]:
submission.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,0.302042
std,61834.358168,0.841117
min,0.000000,0.000000
25%,53549.750000,0.060762
50%,107099.500000,0.089595
75%,160649.250000,0.215351
max,214199.000000,20.000000


In [35]:
submission.to_csv('submission.csv', index=False)

In [ ]:
result: 
test - 1.1
validation - 1.045